<h2>Pre Process Dataset</h2>
<h6>get dataset</br>
split train and test data</br>
spilt every matrix song to one second segment</br></h6>

In [11]:
import numpy as np

DATASET_PATH = "./all.npy"
with open(DATASET_PATH, 'rb') as f:
    data = np.load(f, allow_pickle=True)
data.shape

(13281,)

In [22]:
from sklearn.model_selection import train_test_split

data_train, data_test, _, _ = train_test_split(data, data, test_size=0.2, random_state=42)
data_train.shape, data_test.shape

((10624,), (2657,))

In [26]:
import librosa


In [49]:
sample_rate = 16000
hop_length = int(sample_rate * 0.1)
n_fft = int(sample_rate * 0.2)
DIFF = 1
allowed_duration = 10000
frame_sec_indexes = [0, ] + [librosa.time_to_frames(i, sample_rate, n_fft=n_fft, hop_length=hop_length)
                             for i in range(1, allowed_duration, DIFF)]
frame_sec_indexes[-1]

99989

In [46]:
def split_features(features):

    split_features = list()
    song_indexes = dict()
    for i in range(len(features)):
        pre_len = len(split_features)
        if (len(features[i])) > frame_sec_indexes[-1]:
            print(len(features[i]), i)
        split_feature_i = [features[i][frame_sec_indexes[index]: frame_sec_indexes[index+1]]
                           for index in range(len(frame_sec_indexes)-1)
                           if frame_sec_indexes[index] < len(features[i])]
        # add data to split_feature
        split_features += split_feature_i

        # add data to song_indexes {song3: (start_index, end_index), }
        song_indexes[i] = (pre_len, len(split_features))

    return np.array(split_features), song_indexes

In [50]:
final_train_data, train_song_indexes = split_features(data_train)
final_test_data, test_song_indexes = split_features(data_test)
final_train_data.shape, final_test_data.shape

C:\Users\MJSH\AppData\Local\Temp/ipykernel_14264/860082388.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(split_features), song_indexes


((2415387,), (615124,))